In [34]:
import pickle
import pandas as pd
import numpy as np
import os
import re

In [2]:
# Format:
# Bowler, batsman 
# Runs in numbers/words, out else 0. If wide/no ball then ignore entry
# out
def to_corrrect_format(filepath): 
    df = pd.read_csv(filepath, sep='delimiter',header=None, engine="python") #, names=["Players", "Outcome"]
    numbers_dict = {"zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6}
    data = []
    wkt_taken = 0
    num_over = 0
    prev_bowler = ''
    for index, row in df.iterrows():
        line = row.to_string(index=False)
        list = line.split(",")

        bowler, batsman = list[0].split(" to ")    
        if prev_bowler!=bowler: # what about case senstivity?
            prev_bowler=bowler
            num_over+=1

        find_runs = r'(zero|one|two|three|four|five|six|0|1|2|3|4|5|6)'
        find_extra = r'(wide|no ball)'
        outcome = re.findall(find_runs, list[1], flags=re.IGNORECASE)
        if re.findall("out", list[-1], flags=re.IGNORECASE):
            wkt_taken += 1
            data.append([bowler, batsman, wkt_taken, num_over, 7])
        elif re.findall(find_extra, list[1], flags=re.IGNORECASE):
            continue
        elif outcome:
            # print(outcome)
            runs = 0
            if outcome[0].isdigit():
                runs = int(outcome[0])
            else:
                num = numbers_dict.get(outcome[0].lower(), None)
                if num is not None:
                    runs = num
            data.append([bowler, batsman, wkt_taken, num_over, runs])
        else:
            data.append([bowler, batsman, wkt_taken, num_over, 0])
            # print("yay!")
        # print(list)
    # print(data)
    new_path = filepath.split(".")
    with open(new_path[0]+'.pkl', 'wb') as file:
        pickle.dump(data, file)

In [25]:
# current issues:
#   - Same player last names
#   - Dismissal on extras leads to >6 balls in an over
#   - counting byes as runs

In [31]:
# df = pd.read_csv("T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.csv", sep='delimiter',header=None) #, names=["Players", "Outcome"]
# df.head()

to_corrrect_format("T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.csv")

In [3]:
rootdir = 'T20_matches'
cnt=0
for subdir, dirs, files in os.walk(rootdir):
    # if(cnt>=10):
    #     break
    try: 
        for file in files:
            # if(cnt>=10):
            #     break
            to_corrrect_format(os.path.join(subdir, file))
            # cnt+=1
    except: 
        pass

In [4]:
with open('D:/BTp/btp/T20_matches/australia-vs-bangladesh-11th-match-group-a-412688/textsInnings1.pkl', 'rb') as f:
    data = pickle.load(f)

In [6]:
df = pd.DataFrame(data)

In [7]:
df

,0,1,2,3,4
0,Shafiul,Harris,0,1,2
1,Shafiul,Hussey,1,1,7
2,Shafiul,Hussey,1,1,2
3,Shafiul,Smith,1,1,1
4,Shafiul,Hussey,1,1,1
...,...,...,...,...,...
119,Mortaza,Watson,7,20,1
120,Mortaza,Watson,7,20,0
121,Mortaza,Warner,7,20,1
122,Mortaza,Warner,7,20,0


In [29]:
data1=[]
rootdir = 'T20_matches'
# cnt=0
for subdir, dirs, files in os.walk(rootdir):
    # if(cnt>=20):
    #     break
    try: 
        for file in files:
            if(file[-1]=='v'):
                continue
            # if(cnt>=20):
            #     break
            print(os.path.join(subdir,file ))
            with open(os.path.join(subdir,file), 'rb') as f:
                data1+= (pickle.load(f))
            # cnt+=1
    except: 
        pass

T20_matches\australia-vs-bangladesh-11th-match-group-a-412688\textsInnings1.pkl
T20_matches\australia-vs-bangladesh-11th-match-group-a-412688\textsInnings2.pkl
T20_matches\australia-vs-bangladesh-14th-match-group-f-287866\textsInnings1.pkl
T20_matches\australia-vs-bangladesh-14th-match-group-f-287866\textsInnings2.pkl
T20_matches\australia-vs-england-1st-t20i-446960\textsInnings1.pkl
T20_matches\australia-vs-england-1st-t20i-446960\textsInnings2.pkl
T20_matches\australia-vs-england-2nd-t20i-446961\textsInnings1.pkl
T20_matches\australia-vs-england-2nd-t20i-446961\textsInnings2.pkl
T20_matches\australia-vs-england-9th-match-group-b-287861\textsInnings1.pkl
T20_matches\australia-vs-england-9th-match-group-b-287861\textsInnings2.pkl
T20_matches\australia-vs-england-final-412703\textsInnings1.pkl
T20_matches\australia-vs-england-final-412703\textsInnings2.pkl
T20_matches\australia-vs-england-only-t20i-249227\textsInnings1.pkl
T20_matches\australia-vs-england-only-t20i-249227\textsInnings2.

In [18]:
data1

[['Shafiul', 'Harris', 0, 1, 2],
 ['Shafiul', 'Hussey', 1, 1, 7],
 ['Shafiul', 'Hussey', 1, 1, 2],
 ['Shafiul', 'Smith', 1, 1, 1],
 ['Shafiul', 'Hussey', 1, 1, 1],
 ['Shafiul', 'Smith', 1, 1, 1],
 ['Shafiul', 'Hussey', 1, 1, 1],
 ['Razzak', 'Smith', 1, 2, 0],
 ['Razzak', 'Smith', 1, 2, 0],
 ['Razzak', 'Smith', 1, 2, 6],
 ['Razzak', 'Hussey', 1, 2, 1],
 ['Razzak', 'Smith', 1, 2, 1],
 ['Razzak', 'Smith', 1, 2, 6],
 ['Shakib', 'Smith', 1, 3, 1],
 ['Shakib', 'Hussey', 1, 3, 1],
 ['Shakib', 'Smith', 1, 3, 1],
 ['Shakib', 'Smith', 1, 3, 4],
 ['Shakib', 'Hussey', 1, 3, 1],
 ['Shakib', 'Hussey', 1, 3, 4],
 ['Mortaza', 'Hussey', 1, 4, 1],
 ['Mortaza', 'Hussey', 1, 4, 4],
 ['Mortaza', 'Hussey', 1, 4, 4],
 ['Mortaza', 'Smith', 1, 4, 1],
 ['Mortaza', 'Hussey', 1, 4, 1],
 ['Mortaza', 'Hussey', 1, 4, 6],
 ['Shafiul', 'Smith', 1, 5, 2],
 ['Shafiul', 'Hussey', 1, 5, 1],
 ['Shafiul', 'Smith', 1, 5, 1],
 ['Shafiul', 'Hussey', 1, 5, 1],
 ['Shafiul', 'Hussey', 1, 5, 0],
 ['Shafiul', 'Hussey', 1, 5, 2],
 [

In [30]:
df1= pd.DataFrame(data1)

In [31]:
df1

,0,1,2,3,4
0,Shafiul,Harris,0,1,2
1,Shafiul,Hussey,1,1,7
2,Shafiul,Hussey,1,1,2
3,Shafiul,Smith,1,1,1
4,Shafiul,Hussey,1,1,1
...,...,...,...,...,...
56730,Tanvir,Sibanda,6,20,1
56731,Tanvir,Sibanda,6,20,4
56732,Tanvir,Sibanda,6,20,0
56733,Tanvir,Sibanda,6,20,2


In [32]:
df1=df1.rename(columns={0:"bowler",1:"batsman",2:"wicket",3:"over",4:"outcome"})

In [27]:
df1.columns

Index(['bowler', 'batsman', 'wicket', 'over', 'outcome'], dtype='object')

In [45]:
df1.to_pickle("./df1.pkl")

In [ ]:
#seedha load df1 from df.pkl

In [33]:
batsmen_dict = {batsman: index for index, batsman in enumerate(df1['batsman'].unique())}
print("Dictionary of batsmen names:", batsmen_dict)

Dictionary of batsmen names: {'Harris': 0, 'Hussey': 1, 'Smith': 2, 'White': 3, 'Clarke': 4, 'Haddin': 5, 'Warner': 6, 'Watson': 7, 'Mortaza': 8, 'Razzak': 9, 'Jahurul': 10, 'Shafiul': 11, 'Mushfiqur': 12, 'Md Naeem': 13, 'Shakib': 14, 'Mahmudullah': 15, 'Aftab': 16, 'Ashraful': 17, 'Imrul': 18, 'Farhad': 19, 'Kapali': 20, 'Tamim': 21, 'Nazimuddin': 22, 'Ponting': 23, 'Hayden': 24, 'Gilchrist': 25, 'Finch': 26, 'Woakes': 27, 'Shahzad': 28, 'Swann': 29, 'Bresnan': 30, 'Yardy': 31, 'Morgan': 32, 'Wright': 33, 'PD Collingwood': 34, 'Pietersen': 35, 'Bell': 36, 'Davies': 37, 'Johnson': 38, "O'Keefe": 39, 'Paine': 40, 'Broad': 41, 'Kirtley': 42, 'Schofield': 43, 'AD Mascarenhas': 44, 'Shah': 45, 'Flintoff': 46, 'Maddy': 47, 'Prior': 48, 'Symonds': 49, 'Kieswetter': 50, 'Lumb': 51, 'Anderson': 52, 'Nixon': 53, 'Panesar': 54, 'Lewis': 55, 'Dalrymple': 56, 'Vaughan': 57, 'Joyce': 58, 'Nehra': 59, 'Rohit Sharma': 60, 'Khan': 61, 'Harbhajan': 62, 'Jadeja': 63, 'Pathan': 64, 'Dhoni': 65, 'Yuvraj'

In [35]:
# Create the four-dimensional matrix X
num_batsmen = len(batsmen_dict)
max_over = df1['over'].max() + 1  # Adding 1 to account for zero-based indexing
max_wicket = df1['wicket'].max() + 1  
max_outcome = df1['outcome'].max() + 1  

X = np.zeros((num_batsmen, max_over, max_wicket, max_outcome), dtype=int)

# Populate the matrix X based on DataFrame values
for index, row in df1.iterrows():
    batsman_idx = batsmen_dict[row['batsman']]
    over_idx = row['over']
    wicket_idx = row['wicket']
    outcome_idx = row['outcome']
    
    X[batsman_idx, over_idx, wicket_idx, outcome_idx] += 1

print("Four-dimensional matrix X:")
print(X)

Four-dimensional matrix X:
[[[[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[ 0  0  1 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  ...

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]]

  [[ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   [ 0  0  0 ...  0  0  0]
   ...
   [ 0  0  0 ...  0  0 

In [44]:
X[batsmen_dict['Kohli']][14][2]

array([0, 5, 0, 0, 0, 0, 0, 0])